In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import pywt
import pandas as pd
import holoviews as hv
import param
import panel as pn
from panel.pane import LaTeX
from PIL import Image
hv.extension('bokeh')
import warnings
warnings.filterwarnings('ignore')
import matplotlib.image as mpimg
import time
import matplotlib.pyplot as plt
import numpy.linalg as npl

In [ ]:
Im=np.array(Image.open("./rock.jpg")).astype(float)[:,:,0]
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
np.random.seed(seed=1)
n1,n2=np.shape(Im)
r=np.random.rand(n1,n2)
M=(r<0.5)
M = M*1.0
y=M*Im
pn.Row(hv.Image(Im).opts(**options),hv.Image(y).opts(**options))

In [ ]:
def FiltreMedian(im,masque,vois):
    imrec=np.copy(im)
    n1,n2=np.shape(im)
    K1=int(np.floor(n1/vois))
    K2=int(np.floor(n2/vois))
    for k1 in np.arange(K1):
        for k2 in np.arange(K2):
            n=0
            med=np.median(im[k1*vois:(k1+1)*vois,k2*vois:(k2+1)*vois][masque[k1*vois:(k1+1)*vois,k2*vois:(k2+1)*vois]!=0])
            imrec[k1*vois:(k1+1)*vois,k2*vois:(k2+1)*vois][masque[k1*vois:(k1+1)*vois,k2*vois:(k2+1)*vois]==0]=med
    return(imrec)

In [ ]:
def GradientInpainting(x,b,M):
    g=M*(x-b)
    return g
def SeuillageDouxOndelettes(I,wave,Seuil):
    L=pywt.dwt_max_level(len(I),pywt.Wavelet(wave).dec_len)
    wavelet_coeffs= pywt.wavedecn(I, wave, mode='per', level=L)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(wavelet_coeffs)
    temp=pywt.threshold(arr,Seuil,mode='soft')
    test=pywt.unravel_coeffs(temp, coeff_slices, coeff_shapes, output_format='wavedecn')
    Irec=pywt.waverecn(test, wave,mode='per')
    return Irec
def Normel1Ondelettes(I,wave):
    L=pywt.dwt_max_level(len(I),pywt.Wavelet(wave).dec_len)
    wavelet_coeffs= pywt.wavedecn(I, wave, mode='per', level=L)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(wavelet_coeffs)
    norml1=sum(np.abs(arr))
    return norml1
def F(x,y,M,lam,wave):
    return 0.5*np.linalg.norm(M*x-y)**2+lam*Normel1Ondelettes(x,wave)

In [ ]:
def ForwardBackward_step(x,y,M,step,lam,wave):
    return SeuillageDouxOndelettes(x-step*GradientInpainting(x,y,M),wave,step*lam)

In [ ]:
def ForwardBackward(x0,y,M,step,lam,Niter,wave,epsilon):
    cout=[]
    out=False
    i=0
    x=np.copy(x0)
    while (i<Niter) and out==False:
        xm=np.copy(x)
        x=ForwardBackward_step(x,y,M,step,lam,wave)
        out=npl.norm(xm-x,2)<epsilon
        i+=1
    return np.clip(x,0,255)

In [ ]:
step=0.8
lam=10
Niter=10
wave='sym3'
Ta=350
x0=FiltreMedian(y,M,8)

In [ ]:
imrec=ForwardBackward(x0,y,M,step,lam,Niter,wave,1e-5)
pn.Row(hv.Image(Im).opts(cmap='gray',width=Ta,height=Ta),\
       hv.Image(y).opts(cmap='gray',width=Ta,height=Ta)\
       ,hv.Image(imrec).opts(cmap='gray',width=Ta,height=Ta))

In [ ]:
def FISTA(x,y,M,step,lam,Niter,wave,alpha,epsilon):
    temp=np.copy(x)
    i=0
    out=False
    while i<Niter and out==False:
        i+=1
        xm=np.copy(x)
        x=ForwardBackward_step(temp,y,M,step,lam,wave)
        out=npl.norm(temp-x,2)<epsilon
        temp=x+(i-1)/(i+alpha-1)*(x-xm)
    return x,out

In [ ]:
def FISTA_alamo(x,y,M,step,lam,Niter,wave,alpha,epsilon):
    cout0=F(x,y,M,lam,wave)
    cout=[0]
    temp=np.copy(x)
    i=0
    Ec=False
    seuil=int(Niter/2)
    out=False
    while (Ec==False or i<Niter) and out==False:
        i+=1
        xm=np.copy(x)
        x=ForwardBackward_step(temp,y,M,step,lam,wave)
        out=npl.norm(temp-x,2)<epsilon
        temp=x+(i-1)/(i+alpha-1)*(x-xm)
        if (i>=seuil):
            cout+=[F(x,y,M,lam,wave)]
        if (i>=Niter):
            m=int(i/2)-seuil+1
            Ec=((np.exp(1)*(cout[m]-cout[-1])<=(cout0-cout[m])) and (cout[-1]<=cout0))
    return x, cout[-1],out,i


def restart_FISTA_alamo(x,y,M,step,lam,Niter,wave,alpha,k,epsilon):
    i=0
    cout=[F(x,y,M,lam,wave)]
    x=ForwardBackward_step(x,y,M,step,lam,wave)
    x,cout1,out,k_tot=FISTA_alamo(x,y,M,step,lam,Niter,wave,alpha,epsilon)
    cout+=[cout1]
    km=k
    k=k_tot
    i+=k+1
    out=False
    while (i<Niter) and out==False:
        x=ForwardBackward_step(x,y,M,step,lam,wave)
        x,cout1,out,k_tot=FISTA_alamo(x,y,M,step,lam,np.minimum(k,Niter-i-1),wave,alpha,epsilon)
        cout+=[cout1]
        km=k
        k=k_tot
        i+=k+1
        if (np.exp(1)*(cout[-2]-cout[-1])>(cout[-3]-cout[-2]))and (cout[-3]!=cout[-2]):
            k=2*km
    return np.array(x)

In [ ]:
def restart_FISTA_adap(x,y,M,step,lam,Niter,wave,alpha,C,epsilon):
    i=0
    k=int(2*C)
    F_tab=[F(x,y,M,lam,wave)]
    k_tab=[k]
    x,out=FISTA(x,y,M,step,lam,k,wave,alpha,epsilon)
    i+=k
    F_tab+=[F(x,y,M,lam,wave)]
    k_tab+=[k]
    while (i<Niter) and out==False:
        n_ite=np.minimum(k,Niter-i)
        x,out=FISTA(x,y,M,step,lam,n_ite,wave,alpha,epsilon)
        i+=n_ite
        F_tab+=[F(x,y,M,lam,wave)]
        F_array=np.array(F_tab)
        tab=4/(step*(np.array(k_tab)[:-1]+1)**2)*(F_array[:-2]-F_array[-1])/(F_array[1:-1]-F_array[-1])
        mu_tilde=np.min(tab)
        if (k<=C/np.sqrt(step*mu_tilde)):
            k=2*k
        k_tab+=[k]
    return x

In [ ]:
def restart_grad_alamo(x,y,M,step,lam,Niter,wave,alpha,L,epsilon):
    x=ForwardBackward_step(x,y,M,step,lam,wave)
    temp=np.copy(x)
    gamma=L*npl.norm(x0-x,2)
    i=0
    k=0
    out=False
    xp=ForwardBackward_step(temp,y,M,step,lam,wave)
    while i<Niter and out==False:
        i+=1
        k+=1
        xm=np.copy(x)
        x=np.copy(xp)
        out=npl.norm(temp-x,2)<epsilon
        temp=x+(k-1)/(k+alpha-1)*(x-xm)
        xp=ForwardBackward_step(temp,y,M,step,lam,wave)
        test=L*npl.norm(temp-xp,2)
        if test<=gamma*np.exp(-1):
            gamma=test
            k=0
            i+=1
            xm=np.copy(x)
            x=np.copy(xp)
            xp=ForwardBackward_step(x,y,M,step,lam,wave)
            
    return x

In [ ]:
def FISTA_restart_emp_grad(x,y,M,step,lam,Niter,wave,alpha,epsilon):
    temp=np.copy(x)
    i=0
    verif=False
    out=False
    while (i<Niter) and (verif==False) and (out==False):
        i+=1
        xm=np.copy(x)
        temp_m=np.copy(temp)
        x=ForwardBackward_step(temp,y,M,step,lam,wave)
        out=npl.norm(temp-x,2)<epsilon
        temp=x+(i-1)/(i+alpha-1)*(x-xm)
        verif=np.dot(np.ravel(temp_m-x),np.ravel(x-xm))>0
    return x,out,i


def restart_emp_grad(x,y,M,step,lam,Niter,wave,alpha,epsilon):
    i=0
    out=False
    while (i<Niter) and out==False:
        x,out,ni=FISTA_restart_emp_grad(x,y,M,step,lam,Niter-i,wave,alpha,epsilon)
        i+=ni
    return x

In [ ]:
def FISTA_restart_emp_f(x,y,M,step,lam,Niter,wave,alpha,epsilon):
    cout=[F(x,y,M,lam,wave)]
    temp=np.copy(x)
    i=0
    verif=False
    out=False
    while (i<Niter) and (verif==False) and (out==False):
        i+=1
        xm=np.copy(x)
        x=ForwardBackward_step(temp,y,M,step,lam,wave)
        out=npl.norm(temp-x,2)<epsilon
        temp=x+(i-1)/(i+alpha-1)*(x-xm)
        cout+=[F(x,y,M,lam,wave)]
        verif=cout[-1]>cout[-2]
    return x,out,i


def restart_emp_f(x,y,M,step,lam,Niter,wave,alpha,epsilon):
    i=0
    out=False
    while (i<Niter) and out==False:
        x,out,ni=FISTA_restart_emp_f(x,y,M,step,lam,Niter-i,wave,alpha,epsilon)
        i+=ni
    return x

In [ ]:
Im=np.array(Image.open("./rock.jpg")).astype(float)[:,:,0]
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
np.random.seed(seed=1)
n1,n2=np.shape(Im)
r=np.random.rand(n1,n2)
M=(r<0.5)
M = M*1.0
y=M*Im
step=1
lam=10
alpha=3
Niter=500
wave='db4'
Ta=350
epsilon=1e-6
x0=FiltreMedian(y,M,8)

t0_alamo=time.perf_counter()
x_alamo=restart_FISTA_alamo(x0,y,M,step,lam,Niter,wave,alpha,1,epsilon)
t1_alamo=time.perf_counter()

t0_adap=time.perf_counter()
x_adap=restart_FISTA_adap(x0,y,M,step,lam,Niter,wave,alpha,6.38,epsilon)
t1_adap=time.perf_counter()

t0_g_alamo=time.perf_counter()
x_g_alamo=restart_grad_alamo(x0,y,M,step,lam,Niter,wave,alpha,1/step,epsilon)
t1_g_alamo=time.perf_counter()

t0_g_emp=time.perf_counter()
x_g_emp=restart_emp_grad(x0,y,M,step,lam,Niter,wave,alpha,epsilon)
t1_g_emp=time.perf_counter()

t0_emp=time.perf_counter()
x_emp=restart_emp_f(x0,y,M,step,lam,Niter,wave,alpha,epsilon)
t1_emp=time.perf_counter()

#print("FISTA  ;  Restart by Alamo et al.    ;     Algorithm 2      ;        gradient restart by Alamo et al.")
#print(F(x_FISTA,A,b,rho),F(x_alamo,A,b,rho),F(x_adap,A,b,rho),F(x_g_alamo,A,b,rho))
#print(t1_FISTA-t0_FISTA,t1_alamo-t0_alamo,t1_adap-t0_adap,t1_g_alamo-t0_g_alamo)

Fx_alamo=F(x_alamo,y,M,lam,wave)
Fx_adap=F(x_adap,y,M,lam,wave)
Fx_g_alamo=F(x_g_alamo,y,M,lam,wave)
Fx_g_emp=F(x_g_emp,y,M,lam,wave)
Fx_emp=F(x_emp,y,M,lam,wave)


print("Restart by Alamo et al.;Algorithm 2; Gradient restart by Alamo et al.; Empiric gradient restart;Empiric restart")
print(Fx_alamo,Fx_adap,Fx_g_alamo,Fx_g_emp,Fx_emp)
print(t1_alamo-t0_alamo,t1_adap-t0_adap,t1_g_alamo-t0_g_alamo,t1_g_emp-t0_g_emp,t1_emp-t0_emp)
